<a href="https://colab.research.google.com/github/sanjaynagi/AnoExpress/blob/main/workflow/notebooks/enrichment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install anoexpress -q

In [ ]:
import anoexpress as xpress
import numpy as np
import pandas as pd

### Perform GSEA on custom data

First, a file of AGAP GeneIDs must be uploaded into Colab. It must have the GeneIDs in the first column, with no header. Click on the files tab of Colab on the left hand side, and use the upload button to upload your file. It can be a .txt, .csv, .tsv or .xlsx file.

Then pass the file path of your GeneID file to the `gene_ids` parameter of `xpress.go_hypergeometric()`.

In [ ]:
# gene_id_file = 'my_genes.txt' # replace with your gene ID file name

# my_genes_go_enrichment = xpress.go_hypergeometric(analysis='gamb_colu_arab_fun', gene_ids=gene_id_file) #ignore the analysis argument, it wont be used in this case
# my_genes_go_enrichment

### GO annotations on AnoExpress candidates

In [ ]:
median_go_hypergeo_df = xpress.go_hypergeometric(analysis="gamb_colu_arab_fun", func=np.nanmedian)
median_go_hypergeo_df.query("padj < 0.05")

# Pfam Domains

In [ ]:
median_pfam_hypergeo_df = xpress.pfam_hypergeometric(analysis="gamb_colu_arab_fun", func=np.nanmedian)
median_pfam_hypergeo_df.head(30)

Lots of cool stuff here too - P450s, obps, ORs, GRs, GST, Elongases (ELO), chitin binding seems to be coming up alot.

In [ ]:
### FOR REFERENCE - TO MAP pfam domain to gene family name. Lots of other stuff coming up as significant I have no idea what that is
pfam_domain_names = {'Cytochrome P450s':'p450',
             'GSTs':['GST_N', 'GST_N_3', 'GST_C'],
             'Carboxylesterases': 'COesterase',
             'ABC-transporters':['ABC_membrane', 'ABC_tran'],
             'CSP': 'OS-D',
             'UGTs': 'UDPGT',
             'Odorant binding proteins':'PBP_GOBP',
             'Olfactory receptors':'7tm_6',
             'Ionotropic receptors':['Lig_chan','7tm_1'],
             'Gustatory receptors': '7tm_7',
             'Fatty acid synthases':'ketoacyl-synt',
             'FA Elongase':'ELO',
             'FA desaturase':'FA_desaturase',
             'FA reductase':'NAD_binding_4',
             }

## Kegg pathways

In [ ]:
import pandas as pd
pd.set_option("display.max_rows", 500)

In [ ]:
median_kegg_hypergeo_df = xpress.kegg_hypergeometric(analysis="gamb_colu_arab_fun", func=np.nanmedian)
median_kegg_hypergeo_df.head(50)